In [ ]:
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig

In [12]:
import sys
project_dir = "/data/ephemeral/home/nlp-5/song/"
sys.path.append(
    project_dir
)
from src.dataset.dataset_base import *
from src.dataset.preprocess import *
from src.models.BART import *
from src.trainer.trainer_base import *

In [7]:
import yaml
import os 
config_path = os.path.join(project_dir,
    "src",'configs','config_base_0725125045.yaml'
)
with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

In [ ]:
import torch
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
_, tokenizer = load_tokenizer_and_model_for_train(loaded_config, device)

---------- Load tokenizer & model ----------
---------- Model Name : digit82/kobart-summarization ----------


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartConfig {
  "_name_or_path": "digit82/kobart-summarization",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings":

In [27]:

model_dir = os.path.join(project_dir, "outputs",
    "exp_0725163123", "best"
)
tokenizer = AutoTokenizer.from_pretrained(
     os.path.join(model_dir)
)
model = BartForConditionalGeneration.from_pretrained(
    os.path.join(model_dir)
).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm

# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_test_dataset(config, preprocessor, tokenizer):

    test_file_path = os.path.join(config['general']['data_path'],'test.csv')

    test_data = preprocessor.make_set_as_df(test_file_path,is_train=False)
    test_id = test_data['fname']

    print('-'*150)
    print(f'test_data:\n{test_data["dialogue"][0]}')
    print('-'*150)

    encoder_input_test , decoder_input_test = preprocessor.make_input(test_data,is_test=True)
    print('-'*10, 'Load data complete', '-'*10,)

    test_tokenized_encoder_inputs = tokenizer(encoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False,)
    test_tokenized_decoder_inputs = tokenizer(decoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False,)

    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-'*10, 'Make dataset complete', '-'*10,)

    return test_data, test_encoder_inputs_dataset

# # 추론을 위한 tokenizer와 학습시킨 모델을 불러옵니다.
# def load_tokenizer_and_model_for_test(config,device):
#     print('-'*10, 'Load tokenizer & model', '-'*10,)

#     model_name = config['general']['model_name']
#     ckt_path = config['inference']['ckt_path']
#     print('-'*10, f'Model Name : {model_name}', '-'*10,)
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
#     tokenizer.add_special_tokens(special_tokens_dict)

#     generate_model = BartForConditionalGeneration.from_pretrained(ckt_path)
#     generate_model.resize_token_embeddings(len(tokenizer))
#     generate_model.to(device)
#     print('-'*10, 'Load tokenizer & model complete', '-'*10,)

    return generate_model , tokenizer

# 학습된 모델이 생성한 요약문의 출력 결과를 보여줍니다.
def inference(config, generate_model, tokenizer):
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    # generate_model , tokenizer = load_tokenizer_and_model_for_test(config,device)

    data_path = config['general']['data_path']
    preprocessor = Preprocess(config['tokenizer']['bos_token'], config['tokenizer']['eos_token'])

    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config,preprocessor, tokenizer)
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            generated_ids = generate_model.generate(input_ids=item['input_ids'].to(device),
                            no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                            early_stopping=config['inference']['early_stopping'],
                            max_length=config['inference']['generate_max_length'],
                            num_beams=config['inference']['num_beams'],
                        )
            for ids in generated_ids:
                result = tokenizer.decode(ids)
                summary.append(result)

    # 정확한 평가를 위하여 노이즈에 해당되는 스페셜 토큰을 제거합니다.
    remove_tokens = config['inference']['remove_tokens']
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    output = pd.DataFrame(
        {
            "fname": test_data['fname'],
            "summary" : preprocessed_summary,
        }
    )
    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(os.path.join(result_path, "output.csv"), index=False)

    return output

In [25]:
os.chdir(project_dir)
output = inference(loaded_config, model, tokenizer)

---------- device : cuda ----------
2.1.0
------------------------------------------------------------------------------------------------------------------------------------------------------
test_data:
#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은 해고에 처해질 것입니다. 이 새로운 정책에 대한 어떤 질문이라도 부서장에게 직접 문의하면 됩니다.
#Person2#: 그게 다신가요?
#Person1#: 네. 이 메모를 오후 4시 

100%|██████████| 16/16 [00:23<00:00,  1.44s/it]


In [26]:
output

,fname,summary
0,test_0,더슨 씨는 #Person1# 에게 모든 사무실 통신이 이메일 통신과 공식 메모로...
1,test_1,#Person1# 은 교통 체증에 걸렸다. #Person2# 는 #Person1#...
2,test_2,케이트는 마샤와 히어로가 이혼하려고 한다고 #Person1# 에게 말한다. 그녀...
3,test_3,브라이언은 #Person1# 에게 자신의 생일을 축하하기 위해 파티에 왔다고 말...
4,test_4,#Person2# 는 #Person1# 에게 올림픽 스타디움에 5000개의 좌석이...
...,...,...
494,test_495,찰리는 아빠를 데리러 가야하기 때문에 잭과 함께 새 게임을 하고 싶어한다. 잭은...
495,test_496,#Person2# 는 컨트리 음악에 관심을 가지게 된 계기와 라디오 방송국에서 일...
496,test_497,"앨리스는 #Person1# 에게 기계를 어떻게 사용하는지 알려주고, 기계가 너무..."
497,test_498,스티브와 매튜는 계약이 다음 달에 끝나기 때문에 새로운 집을 찾고 있다. 그들은...
